In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-056_az8d
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-056_az8d
     |████████████████████████████████| 204kB 3.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 327kB 14.7MB/s 
     |████████████████████████████████| 337kB 19.0MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 983kB 22.1MB/s 
     |████████████████████████████████| 5.6MB 11.8MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 8.4MB 34.8MB/s 
     |████████████████████████████████| 348kB 30.2MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.0

In [ ]:
from haystack import Finder

from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

# initiating ElasticSearch
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")


06/02/2021 07:45:00 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.114s]
06/02/2021 07:45:00 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.423s]
06/02/2021 07:45:01 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.264s]


In [ ]:
data_dir = '/content/drive/MyDrive/Textfiles'

In [ ]:
dicts = convert_files_to_dicts(dir_path=data_dir, clean_func=clean_wiki_text, split_paragraphs=True)


06/02/2021 07:45:57 - INFO - haystack.preprocessor.utils -   Converting /content/drive/MyDrive/Textfiles/First.txt
06/02/2021 07:45:58 - INFO - haystack.preprocessor.utils -   Converting /content/drive/MyDrive/Textfiles/Second.txt
06/02/2021 07:45:58 - INFO - haystack.preprocessor.utils -   Converting /content/drive/MyDrive/Textfiles/Third.txt
06/02/2021 07:45:59 - INFO - haystack.preprocessor.utils -   Converting /content/drive/MyDrive/Textfiles/Fourth.txt
06/02/2021 07:46:00 - INFO - haystack.preprocessor.utils -   Converting /content/drive/MyDrive/Textfiles/Fifth.txt


In [ ]:
document_store.write_documents(dicts)


06/02/2021 07:46:03 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.315s]


In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)


In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True, context_window_size=500)


06/02/2021 07:46:10 - INFO - farm.utils -   Using device: CUDA 
06/02/2021 07:46:10 - INFO - farm.utils -   Number of GPUs: 1
06/02/2021 07:46:10 - INFO - farm.utils -   Distributed Training: False
06/02/2021 07:46:10 - INFO - farm.utils -   Automatic Mixed Precision: None
06/02/2021 07:46:11 - INFO - filelock -   Lock 140182884748944 acquired on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


06/02/2021 07:46:12 - INFO - filelock -   Lock 140182884748944 released on /root/.cache/huggingface/transformers/c40d0abb589629c48763f271020d0b1f602f5208c432c0874d420491ed37e28b.122ed338b3591c07dba452777c59ff52330edb340d3d56d67aa9117ad9905673.lock


06/02/2021 07:46:13 - INFO - filelock -   Lock 140183175639632 acquired on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


06/02/2021 07:46:31 - INFO - filelock -   Lock 140183175639632 released on /root/.cache/huggingface/transformers/eac3273a8097dda671e3bea1db32c616e74f36a306c65b4858171c98d6db83e9.084aa7284f3a51fa1c8f0641aa04c47d366fbd18711f29d0a995693cfdbc9c9e.lock


Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
06/02/2021 07:46:45 - INFO - filelock -   Lock 140182872218960 acquired on /root/.cache/huggingface/transformers/81c80edb4c6cefa5cae64ccfdb34b3b309ecaf60da99da7cd1c17e24a5d36eb5.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


...

06/02/2021 07:46:51 - INFO - filelock -   Lock 140182871078352 acquired on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


06/02/2021 07:46:52 - INFO - filelock -   Lock 140182871078352 released on /root/.cache/huggingface/transformers/c9d2c178fac8d40234baa1833a3b1903d393729bf93ea34da247c07db24900d0.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock


06/02/2021 07:46:52 - INFO - filelock -   Lock 140182871078800 acquired on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock


06/02/2021 07:46:53 - INFO - filelock -   Lock 140182871078800 released on /root/.cache/huggingface/transformers/e8a600814b69e3ee74bb4a7398cc6fef9812475010f16a6c9f151b2c2772b089.451739a2f3b82c3375da0dfc6af295bedc4567373b171f514dd09a4cc4b31513.lock
06/02/2021 07:46:53 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
06/02/2021 07:46:53 - INFO - farm.utils -   Using device: CUDA 
06/02/2021 07:46:53 - INFO - farm.utils -   Number of GPUs: 1
06/02/2021 07:46:53 - INFO - farm.utils -   Distributed Training: False


06/02/2021 07:46:53 - INFO - farm.utils -   Automatic Mixed Precision: None
06/02/2021 07:46:54 - INFO - farm.infer -   Got ya 2 parallel workers to do inference ...
06/02/2021 07:46:54 - INFO - farm.infer -    0    0 
06/02/2021 07:46:54 - INFO - farm.infer -   /w\  /w\
06/02/2021 07:46:54 - INFO - farm.infer -   /'\  / \
06/02/2021 07:46:54 - INFO - farm.infer -     


In [ ]:
reader1 = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)


06/02/2021 07:47:07 - INFO - filelock -   Lock 140182848691728 acquired on /root/.cache/huggingface/transformers/ab70e5f489e00bb2df55e4bae145e9b1c7dc794cfa0fd8228e1299d400613429.f3874c2af5400915dc843c97f502c5d30edc728e5ec3b60c4bd6958e87970f75.lock


06/02/2021 07:47:07 - INFO - filelock -   Lock 140182848691728 released on /root/.cache/huggingface/transformers/ab70e5f489e00bb2df55e4bae145e9b1c7dc794cfa0fd8228e1299d400613429.f3874c2af5400915dc843c97f502c5d30edc728e5ec3b60c4bd6958e87970f75.lock


06/02/2021 07:47:08 - INFO - filelock -   Lock 140182878509456 acquired on /root/.cache/huggingface/transformers/b00ff18397f70f871bd8f11949a3c5ffd5fb18fd6d4e3df947dc386950b8d59d.69a963759b72d26fb77afa9b7d43c9107b99dfe7ca78af52e0237c8d001c7dcf.lock


06/02/2021 07:47:18 - INFO - filelock -   Lock 140182878509456 released on /root/.cache/huggingface/transformers/b00ff18397f70f871bd8f11949a3c5ffd5fb18fd6d4e3df947dc386950b8d59d.69a963759b72d26fb77afa9b7d43c9107b99dfe7ca78af52e0237c8d001c7dcf.lock


06/02/2021 07:47:21 - INFO - filelock -   Lock 140182848688208 acquired on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock


06/02/2021 07:47:21 - INFO - filelock -   Lock 140182848688208 released on /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock


06/02/2021 07:47:22 - INFO - filelock -   Lock 140182871223760 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


06/02/2021 07:47:24 - INFO - filelock -   Lock 140182871223760 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


06/02/2021 07:47:24 - INFO - filelock -   Lock 140182878611024 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


06/02/2021 07:47:25 - INFO - filelock -   Lock 140182878611024 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [ ]:
finder = Finder(reader, retriever)


06/02/2021 07:47:49 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [ ]:
finder1 = Finder(reader1, retriever)


06/02/2021 07:47:52 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [ ]:
prediction = finder.get_answers(question="draper", top_k_retriever=10, top_k_reader=2)


06/02/2021 08:26:33 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
06/02/2021 08:26:33 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.021s]
06/02/2021 08:26:33 - INFO - haystack.finder -   Got 3 candidates from retriever
06/02/2021 08:26:33 - INFO - haystack.finder -   Reader is looking for detailed answer in 20285 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.83 Batches/s]
06/02/2021 08:26:35 - ERROR - farm.modeling.predictions -   Both start and end offset

In [ ]:
prediction1 = finder1.get_answers(question="draper", top_k_retriever=10, top_k_reader=2)


06/02/2021 08:26:36 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
06/02/2021 08:26:36 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.018s]
06/02/2021 08:26:36 - INFO - haystack.finder -   Got 3 candidates from retriever
06/02/2021 08:26:36 - INFO - haystack.finder -   Reader is looking for detailed answer in 20285 chars ...


In [ ]:
print(prediction)


{'query': 'draper', 'no_ans_gap': -6.840569734573364, 'answers': [{'answer': 'carpenter', 'score': -1.4071099758148193, 'probability': 0.02537844330072403, 'context': "ied (by sibly related to Lawrence de Anne. vere. Draper. Citizen 1395. , Lucy wife of (I. by 1360; d. by 1378). Alfred the butcher (fl. by 1264-6). ?Part 300-3/329. Alfred the painter (fl. by 1285; ?d. by 1300). Aluredus pictor. 7. Painter 1285, carpenter by 1300. Algar(r), John (fl. 1482-6). 268, 472. Fuller. Auditor for commons 1484-5 (CA). Inhabited ?part 961. Subsidy 4s. (1327), 3s. (1332), 8s. (1340). Lent Crown share £30 c.1340 (Cal Pat R 1338-40, p. 450). Alderman of St. John's Street (13", 'offset_start': 246, 'offset_end': 255, 'offset_start_in_doc': 2894, 'offset_end_in_doc': 2903, 'document_id': 'cb032a6db58111448ff85769f9531714', 'meta': {'name': 'Third.txt'}}, {'answer': 'Richard Harper of London 1414 (Cal Close R 1413-19, p. 185). Aulnager in city and soke 1399 (Fine R 1399-1405, p. 38); bailiff 2 (1404-5);

In [ ]:
print(prediction1)


{'query': 'draper', 'answers': [{'answer': 'Ablyndon, John, v. John Abyn(g)done', 'context': 'illiam As: part 629. Ace, William (f1. 1333, by 1346), v. William As. Ablyndon, John, v. John Abyn(g)done. Abodeston, Henry, v. Henry Abbotestone. Abraham (fl. c. 1222-73). ?P', 'offset_start': 3163, 'offset_end': 3198, 'probability': 0.5775532722473145, 'score': None, 'document_id': 'b0af4135d730767d82320000e70a88a7', 'meta': {'name': 'First.txt'}}, {'answer': None, 'score': 0.6604955196380615, 'probability': 0.5206287683645802, 'context': None, 'offset_start': 0, 'offset_end': 0, 'document_id': None, 'meta': {}}], 'question': 'draper'}


In [ ]:
question = "who is John "
number_of_answers_to_fetch = 2

#prediction = finder.get_answers(question=question, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch)
print(f"Question: {prediction['question']}")
print("\n")
for i in range(number_of_answers_to_fetch):
    print(f"#{i+1}")
    print(f"Answer: {prediction['answers'][i]['answer']}")
    print(f": {prediction['answers'][i]['meta']['name']}")
    print(f"Context: {prediction['answers'][i]['context']}")
    print('\n\n')

Question: draper


#1
Answer: carpenter
: Third.txt
Context: ied (by sibly related to Lawrence de Anne. vere. Draper. Citizen 1395. , Lucy wife of (I. by 1360; d. by 1378). Alfred the butcher (fl. by 1264-6). ?Part 300-3/329. Alfred the painter (fl. by 1285; ?d. by 1300). Aluredus pictor. 7. Painter 1285, carpenter by 1300. Algar(r), John (fl. 1482-6). 268, 472. Fuller. Auditor for commons 1484-5 (CA). Inhabited ?part 961. Subsidy 4s. (1327), 3s. (1332), 8s. (1340). Lent Crown share £30 c.1340 (Cal Pat R 1338-40, p. 450). Alderman of St. John's Street (13



#2
Answer: Richard Harper of London 1414 (Cal Close R 1413-19, p. 185). Aulnager in city and soke 1399 (Fine R 1399-1405, p. 38); bailiff 2 (1404-5); elected one of the 24 (1406) (CR); MP 1407, 1411. ~~
: Fourth.txt
Context: isc 1398, no. 407); William Barelle of New Alresford owed him money 1410 (Cal Close R 1409- 13, p. 109). General release of all personal actions to him by Richard Harper of London 1414 (Cal Close R 1413-19, p. 

In [ ]:
question = "baker"
number_of_answers_to_fetch = 2

#prediction = finder.get_answers(question=question, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch)
print(f"Question: {prediction1['question']}")
print("\n")
for i in range(number_of_answers_to_fetch):
    print(f"#{i+1}")
    print(f"Answer: {prediction1['answers'][i]['answer']}")
    print(f"Research Paper: {prediction['answers'][i]['meta']['name']}")
    print(f"Context: {prediction1['answers'][i]['context']}")
    print('\n\n')

Question: draper


#1
Answer: Ablyndon, John, v. John Abyn(g)done
Research Paper: Third.txt
Context: illiam As: part 629. Ace, William (f1. 1333, by 1346), v. William As. Ablyndon, John, v. John Abyn(g)done. Abodeston, Henry, v. Henry Abbotestone. Abraham (fl. c. 1222-73). ?P



#2
Answer: None
Research Paper: Fourth.txt
Context: None





In [ ]:
print_answers(prediction, details="minimal")


[   {   'answer': 'carpenter',
        'context': 'ied (by sibly related to Lawrence de Anne. vere. Draper. '
                   'Citizen 1395. , Lucy wife of (I. by 1360; d. by 1378). '
                   'Alfred the butcher (fl. by 1264-6). ?Part 300-3/329. '
                   'Alfred the painter (fl. by 1285; ?d. by 1300). Aluredus '
                   'pictor. 7. Painter 1285, carpenter by 1300. Algar(r), John '
                   '(fl. 1482-6). 268, 472. Fuller. Auditor for commons 1484-5 '
                   '(CA). Inhabited ?part 961. Subsidy 4s. (1327), 3s. (1332), '
                   '8s. (1340). Lent Crown share £30 c.1340 (Cal Pat R '
                   "1338-40, p. 450). Alderman of St. John's Street (13"},
    {   'answer': 'Richard Harper of London 1414 (Cal Close R 1413-19, p. '
                  '185). Aulnager in city and soke 1399 (Fine R 1399-1405, p. '
                  '38); bailiff 2 (1404-5); elected one of the 24 (1406) (CR); '
                  'MP 1407, 1411

In [ ]:
print_answers(prediction1, details="minimal")

[   {   'answer': 'Ablyndon, John, v. John Abyn(g)done',
        'context': 'illiam As: part 629. Ace, William (f1. 1333, by 1346), v. '
                   'William As. Ablyndon, John, v. John Abyn(g)done. '
                   'Abodeston, Henry, v. Henry Abbotestone. Abraham (fl. c. '
                   '1222-73). ?P'},
    {'answer': None, 'context': None}]


Open online sources
First names
https://www.behindthename.com/
http://www.cyberspacei.com/englishwiz/library/names/etymology_of_first_names.htm
https://www.familyeducation.com/baby-names/browse-origin/first-name
https://en.geneanet.org/first-name/
https://www.babynamewizard.com/
www.krysstal.com
https://s-gabriel.org/names/index.shtml
https://www.nordicnames.de/wiki/Main_Page
http://www.20000-names.com/

Last names – Family names – Surnames
https://www.familyeducation.com/baby-names/browse-origin/surname
https://en.geneanet.org/surnames/
https://www.surnamedb.com/

Occupations
Mason - https://en.wikipedia.org/wiki/History_of_Freemasonry
Butcher - https://en.wikipedia.org/wiki/Butcher
http://arcana.wikidot.com/list-of-medieval-european-professions
https://hkcarms.tripod.com/occ.html
http://medieval.stormthecastle.com/medieval-jobs.htm
https://www.medievalists.net/2014/06/jobs-middle-ages/
